# Test model

### original file

https://github.com/openai/gpt-2-output-dataset/blob/master/baseline.py

## Before running this notebook

1. Create /output folder
   1. Insert all crawled dataset(csv)
   1. Rename them as same with GPT dataset files.
1. Create /log folder

## Difference between baseline

- Using Word2Vec instead of TF-IDF Vectorizer
- Using Gridsearch to find best parameter

In [1]:
!pip install gensim
!pip install scikit-learn

In [76]:
# import packages

import os
import csv
import json
import time

import numpy as np

from scipy import sparse

from sklearn.model_selection import PredefinedSplit, GridSearchCV, train_test_split, KFold
from sklearn.linear_model import LogisticRegression

from gensim.models import Word2Vec

In [77]:
# create tokenizer
# example code from https://github.com/SKT-AI/KoGPT2

from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token="</s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [78]:
SHORT_MODE = False

In [79]:
# load data + preprocessing

def load_data(data_dir, crawled_dir, source, max_len=250):
    path = os.path.join(data_dir, "{}.csv".format(source))
    crawled_path = os.path.join(crawled_dir, "{}.csv".format(source))
    dataset = list(csv.reader(open(path, encoding="utf8")))
    crawled_dataset = list(csv.reader(open(crawled_path, encoding="cp949")))
    
    try:
        path2 = os.path.join(data_dir, "{}2.csv".format(source))
        dataset2 = list(csv.reader(open(path2, encoding="utf8")))
        dataset.extend(dataset2)
    except:
        pass
    
    n = len(dataset)
    
    length = min(max_len, min(50, n) if SHORT_MODE else n)

    texts = []
    labels = [1, 0] * length

    for data in dataset[:length]:
        idx = int(round(float(data[0])))
        tokens = tokenizer.tokenize(data[5])
        texts.append(' '.join(tokens))
        tokens = tokenizer.tokenize(crawled_dataset[idx][4])
        texts.append(' '.join(tokens))
    return texts, labels

In [80]:
# main function

def main(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
    train_test_ratio=0.1,
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic, 750)
        texts_list.extend(texts)
        labels_list.extend(labels)

    texts_train, texts_test, labels_train, labels_test = train_test_split(
        texts_list, labels_list, test_size=train_test_ratio, random_state=42, shuffle=True,
    )
    n_train, n_test = len(texts_train), len(texts_test)
    
    w2v_model = Word2Vec(sentences=texts_list, vector_size=512, window=9, min_count=5, workers=4, sg=0)
    train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
    test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

    model = LogisticRegression(max_iter=20)
    params = {'C': [1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16, 32, 64]}
    split = PredefinedSplit([-1]*n_train+[0]*n_test)
    search = GridSearchCV(model, params, cv=split, refit=False)
    search.fit(sparse.vstack([train_features, test_features]), labels_train + labels_test)
    model = model.set_params(**search.best_params_)
    model.fit(train_features, labels_train)
    test_accuracy = model.score(test_features, labels_test) * 100.0

    result = model.predict(test_features)
    result_proba = model.predict_proba(test_features)
    result_log_proba = model.predict_log_proba(test_features)
    kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
    for res, pred in zip(result, labels_test):
        if res == 1:
            kind["tp" if res == pred else "fp"] += 1
        else:
            kind["tn" if res == pred else "fn"] += 1
    precision = kind["tp"] / (kind["tp"] + kind["fp"])
    recall = kind["tp"] / (kind["tp"] + kind["fn"])

    ce_loss = 0
    for label, value in zip(labels_test, result_log_proba):
        ce_loss -= label * value[1] + (1 - label) * value[0]
    ce_loss /= len(labels_test)
    data = {
        "test_accuracy": test_accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "F_score": 2 * precision * recall / (precision + recall),
        "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
        "ce_loss": ce_loss,
        "param": search.best_params_,
        "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
    }
    print(data)
    json.dump(data, open(os.path.join(log_dir, f"result_{str(round(time.time() * 1000))}.json"), "w"), indent=4)

In [81]:
# main function

def main_overfit(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
    train_test_ratio=0.1,
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic, 750)
        texts_list.extend(texts)
        labels_list.extend(labels)

    texts_train, texts_test, labels_train, labels_test = train_test_split(
        texts_list, labels_list, test_size=train_test_ratio, random_state=42, shuffle=True,
    )
    
    w2v_model = Word2Vec(sentences=texts_list, vector_size=512, window=9, min_count=5, workers=4, sg=0)
    train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
    test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

    model = LogisticRegression(penalty=None, max_iter=100000)
    model.fit(train_features, labels_train)
    test_accuracy = model.score(test_features, labels_test) * 100.0

    result = model.predict(test_features)
    result_proba = model.predict_proba(test_features)
    result_log_proba = model.predict_log_proba(test_features)
    kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
    for res, pred in zip(result, labels_test):
        if res == 1:
            kind["tp" if res == pred else "fp"] += 1
        else:
            kind["tn" if res == pred else "fn"] += 1
    precision = kind["tp"] / (kind["tp"] + kind["fp"])
    recall = kind["tp"] / (kind["tp"] + kind["fn"])

    ce_loss = 0
    for label, value in zip(labels_test, result_log_proba):
        ce_loss -= label * value[1] + (1 - label) * value[0]
    ce_loss /= len(labels_test)
    data = {
        "test_accuracy": test_accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "F_score": 2 * precision * recall / (precision + recall),
        "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
        "ce_loss": ce_loss,
        "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
    }
    print(data)
    json.dump(data, open(os.path.join(log_dir, f"result_{str(round(time.time() * 1000))}_overfit.json"), "w"), indent=4)

In [82]:
# main function

def main_kfold(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
    train_test_ratio=0.1,
    window_size=9,
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic, 750)
        texts_list.extend(texts)
        labels_list.extend(labels)

    texts_list_train, texts_list_test, labels_list_train, labels_list_test = train_test_split(
        texts_list, labels_list, test_size=train_test_ratio, random_state=42, shuffle=True,
    )
    
    w2v_model = Word2Vec(sentences=texts_list_train, vector_size=512, window=window_size, min_count=5, workers=4, sg=0)
    
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    models = []
    param_list = []
    
    for idx, (train, test) in enumerate(kf.split(texts_list_train)):
        texts_train, texts_test = [texts_list_train[x] for x in train], [texts_list_train[x] for x in test]
        labels_train, labels_test = [labels_list_train[x] for x in train], [labels_list_train[x] for x in test]

        n_train, n_test = len(train), len(test)
        
        train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
        test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

        model = LogisticRegression(max_iter=100)
        params = {'C': [1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16, 32, 64]}
        split = PredefinedSplit([-1]*n_train+[0]*n_test)
        search = GridSearchCV(model, params, cv=split, refit=False)
        search.fit(sparse.vstack([train_features, test_features]), labels_train + labels_test)
        model = model.set_params(**search.best_params_)
        model.fit(train_features, labels_train)
        test_accuracy = model.score(test_features, labels_test) * 100.0
        
        models.append(model)
        param_list.append(search.best_params_)

        result = model.predict(test_features)
        result_proba = model.predict_proba(test_features)
        result_log_proba = model.predict_log_proba(test_features)
        kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
        for res, pred in zip(result, labels_test):
            if res == 1:
                kind["tp" if res == pred else "fp"] += 1
            else:
                kind["tn" if res == pred else "fn"] += 1
        precision = kind["tp"] / (kind["tp"] + kind["fp"])
        recall = kind["tp"] / (kind["tp"] + kind["fn"])

        ce_loss = 0
        for label, value in zip(labels_test, result_log_proba):
            ce_loss -= label * value[1] + (1 - label) * value[0]
        ce_loss /= len(labels_test)
        data = {
            "test_accuracy": test_accuracy,
            "test_precision": precision,
            "test_recall": recall,
            "F_score": 2 * precision * recall / (precision + recall),
            "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
            "ce_loss": ce_loss,
            "param": search.best_params_,
            "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
        }
        print(data)
        json.dump(data, open(os.path.join(log_dir, f"result_{window_size}_{str(round(time.time() * 1000))}_{idx}.json"), "w"), indent=4)
    
    # soft voting
    results = []
    results_proba = []
    results_log_proba = []
    test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_list_test]
    for model in models:
        result = model.predict(test_features)
        result_proba = model.predict_proba(test_features)
        result_log_proba = model.predict_log_proba(test_features)
        
        results.append(result)
        results_proba.append(result_proba)
        results_log_proba.append(result_log_proba)
    
    results = np.array(results)
    results_proba = np.array(results_proba)
    results_log_proba = np.array(results_log_proba)
    
    proba = np.mean(results_proba, axis=0)
    result = np.argmax(proba, axis=1)
    log_proba = np.log(proba)
    
    kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
    for res, pred in zip(result, labels_list_test):
        if res == 1:
            kind["tp" if res == pred else "fp"] += 1
        else:
            kind["tn" if res == pred else "fn"] += 1
    precision = kind["tp"] / (kind["tp"] + kind["fp"])
    recall = kind["tp"] / (kind["tp"] + kind["fn"])

    ce_loss = 0
    for label, value in zip(labels_list_test, log_proba):
        ce_loss -= label * value[1] + (1 - label) * value[0]
    ce_loss /= len(labels_list_test)
    data = {
        "test_accuracy": test_accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "F_score": 2 * precision * recall / (precision + recall),
        "mse_loss": np.sum(np.array(labels_list_test) - proba[:, 1]) ** 2 / len(labels_list_test),
        "ce_loss": ce_loss,
        "param": param_list,
        "label_and_result": list(zip(labels_list_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
    }
    print("Final:", data)
    json.dump(data, open(os.path.join(log_dir, f"result_{window_size}_{str(round(time.time() * 1000))}_total.json"), "w"), indent=4)

In [83]:
# run main function

# main(train_test_ratio=0.2)
# main_overfit(train_test_ratio=0.2)
for i in range(1, 21, 2):
    main_kfold(window_size=i)

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'test_accuracy': 89.81481481481481, 'test_precision': 0.9124839124839125, 'test_recall': 0.8796526054590571, 'F_score': 0.8957675300063171, 'mse_loss': 0.005345949911558648, 'ce_loss': 0.28744912834526753, 'param': {'C': 64}, 'label_and_result': [(1, 1, 0.7231700958833903), (1, 1, 0.7906383345267154), (0, 0, 0.07988135325430082), (0, 0, 0.25988768525923456), (1, 1, 0.7927104769947505), (1, 1, 0.9092886134969341), (1, 1, 0.9760616319856004), (1, 1, 0.9342566873727205), (0, 1, 0.5408047196025969), (0, 0, 0.3331874844447373), (1, 1, 0.9724318089891464), (1, 1, 0.8365745477537168), (1, 1, 0.9621628744968647), (1, 1, 0.9713629772504905), (0, 0, 0.15658707013628972), (0, 0, 0.07279815095366506), (1, 1, 0.9157960100689939), (0, 0, 0.23552272447214845), (1, 0, 0.03386069557325455), (1, 1, 0.8509968761339569), (1, 1, 0.6237515994283455), (0, 0, 0.23392422974133334), (1, 1, 0.8491957603429366), (1, 1, 0.9027979879200014), (1, 1, 0.7633796967025085), (0, 1, 0.9242811634107809), (1, 1, 0.52766176

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

{'test_accuracy': 91.17283950617285, 'test_precision': 0.933420365535248, 'test_recall': 0.885997521685254, 'F_score': 0.9090909090909091, 'mse_loss': 0.0035945200862911334, 'ce_loss': 0.27750427160945834, 'param': {'C': 64}, 'label_and_result': [(1, 1, 0.9436583031563782), (1, 1, 0.8552982062990068), (1, 1, 0.9730054667498977), (1, 1, 0.9113577390435553), (0, 0, 0.17250235218840718), (0, 0, 0.3874373107268238), (1, 1, 0.9364821868006963), (0, 0, 0.022405464624133373), (1, 1, 0.8052956822926937), (0, 0, 0.24616572477906015), (0, 0, 0.24528410724497224), (1, 1, 0.8953182703372916), (0, 0, 0.3771414465192881), (0, 0, 0.4160644782746862), (0, 0, 0.19997995514216832), (0, 1, 0.656209500121084), (0, 0, 0.06641480299445937), (1, 1, 0.8785440087684235), (0, 0, 0.26422886685054586), (0, 0, 0.14734333651945666), (0, 0, 0.08140127577518368), (0, 0, 0.12651710489531626), (0, 0, 0.18019307166032641), (1, 1, 0.9853687950152475), (1, 1, 0.9749389059875463), (0, 0, 0.17543148682544285), (1, 1, 0.8798

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

...

{'test_accuracy': 90.67901234567901, 'test_precision': 0.9281984334203656, 'test_recall': 0.8810408921933085, 'F_score': 0.9040050858232677, 'mse_loss': 0.0002485158101734194, 'ce_loss': 0.26835910509640537, 'param': {'C': 64}, 'label_and_result': [(1, 1, 0.969015112540647), (1, 1, 0.9600916239418471), (1, 1, 0.9208564641682001), (1, 1, 0.9347450190146673), (0, 0, 0.11241987612546901), (0, 0, 0.36363343212335575), (1, 1, 0.9547253568993862), (0, 0, 0.029428166342283343), (1, 1, 0.8431546182834597), (0, 1, 0.5766759969443709), (0, 0, 0.11452434753494556), (1, 1, 0.8765540668523799), (0, 0, 0.29954490846093673), (0, 0, 0.29303520503350716), (0, 0, 0.19222429525276577), (0, 1, 0.5594917243754065), (0, 0, 0.3720432558410724), (1, 1, 0.920302464416522), (0, 0, 0.15534458565441106), (0, 0, 0.2319105153679588), (0, 0, 0.0644649324536101), (0, 0, 0.08266900223213516), (0, 0, 0.23404889194682227), (1, 1, 0.9910346815344206), (1, 1, 0.9723997049480843), (0, 0, 0.15540995038367142), (1, 1, 0.8798

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

{'test_accuracy': 89.32098765432099, 'test_precision': 0.9146666666666666, 'test_recall': 0.8628930817610063, 'F_score': 0.8880258899676375, 'mse_loss': 1.8498562828326409e-06, 'ce_loss': 0.29428182167063105, 'param': {'C': 64}, 'label_and_result': [(0, 0, 0.01694757922431966), (1, 1, 0.9283028017941541), (1, 0, 0.02949719480468776), (1, 0, 0.2724086861310855), (0, 0, 0.04241055527976861), (0, 0, 0.2172401999521254), (1, 1, 0.8544321130834203), (1, 1, 0.5738452909976753), (0, 0, 0.10153386647591219), (0, 0, 0.11591381412754981), (1, 1, 0.9619454069835569), (0, 0, 0.4267426631446033), (1, 1, 0.9522918040445133), (0, 1, 0.5547951338414747), (0, 0, 0.08823505995189584), (0, 0, 0.04268557929456414), (0, 0, 0.16703516864826967), (1, 1, 0.8484959329335162), (1, 1, 0.9788175087781478), (1, 1, 0.9559443507448956), (0, 0, 0.0699613667929674), (1, 1, 0.8498686345223245), (0, 1, 0.7825197058166372), (0, 0, 0.23529127357864107), (1, 1, 0.8096773012770818), (1, 1, 0.982654219832281), (0, 1, 0.56827

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

{'test_accuracy': 90.4320987654321, 'test_precision': 0.9169859514687101, 'test_recall': 0.8886138613861386, 'F_score': 0.9025769956002515, 'mse_loss': 0.06048929402185907, 'ce_loss': 0.27503900762960043, 'param': {'C': 64}, 'label_and_result': [(1, 1, 0.989249275643222), (0, 0, 0.13982414676317717), (0, 0, 0.012727224593315777), (0, 0, 0.05900478724538274), (1, 1, 0.9441428061851692), (1, 0, 0.23882252172011736), (0, 0, 0.30135658855628117), (1, 1, 0.9715753575667425), (0, 0, 0.075107398439627), (0, 0, 0.4702814910761375), (1, 0, 0.019448351916668803), (0, 0, 0.11867580619228119), (0, 0, 0.025889209170064986), (0, 0, 0.1415446595758647), (1, 1, 0.9016306493438593), (1, 1, 0.9810605618829876), (1, 1, 0.9853649943216656), (1, 1, 0.5074269494677011), (0, 0, 0.05129769112576239), (1, 1, 0.7496202459834019), (1, 1, 0.9286470292333756), (0, 0, 0.13258137073311022), (0, 0, 0.14098613809843444), (0, 1, 0.598740596076598), (0, 0, 0.05533229900268228), (0, 0, 0.12465525609171689), (1, 1, 0.8782